# Importing necessary libraries and data

In [ ]:
#run this cell to import necessary libraries
import numpy as np
import pandas as pd
import os
import random
import itertools
import cv2 as cv
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
import glob
import json

!pip install -U pillow
from PIL import Image

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import sklearn.metrics as metrics
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from keras import layers, Input
from tensorflow.keras.preprocessing.image import load_img,img_to_array

from keras.utils import np_utils, normalize
from keras.models import Model, Sequential, load_model
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Reshape, Dropout, Conv2DTranspose, Activation, Concatenate, MaxPooling2D, UpSampling2D, GlobalAveragePooling2D
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ProgbarLogger
from keras.applications import EfficientNetB7

from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.datasets import make_classification

from google.colab.patches import cv2_imshow

!pip install -U ultralytics
from ultralytics import YOLO

!pip install ray[tune]
!pip install hyperopt==0.2.5

import ray
from hyperopt import hp
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search.hyperopt import HyperOptSearch

import torch
import torch.nn as nn
import torch.nn.functional as F

import albumentations as A

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


#Section 1: Preprocessing

#### Creating multiple functions for augmentation, each with paramenters for fine-tuning

In [ ]:
#defines functions for augmentation

#modifies dimensions of the image from original using OpenCV method resize()
def crop(img, newSize):
  width, height, ______ = img.shape
  if width == height:
    return cv.resize(img, newSize)
  length = min(width, height)
  left = (width - length) // 2
  top = (height - length) // 2
  right = (width + length) // 2
  bottom = (height + length) // 2
  return cv.resize(img[left:right, top:bottom, :], newSize)

#randomizes brightness of the image, converts to HSV and multiplies all pixels by a random brightness value
def brightness(img, low, high):
  value = random.uniform(low, high)
  hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
  hsv = np.array(hsv, dtype = np.float64)
  hsv[:,:,1] = hsv[:,:,1]*value
  hsv[:,:,1][hsv[:,:,1]>255]  = 255
  hsv[:,:,2] = hsv[:,:,2]*value
  hsv[:,:,2][hsv[:,:,2]>255]  = 255
  hsv = np.array(hsv, dtype = np.uint8)
  img = cv.cvtColor(hsv, cv.COLOR_HSV2RGB)
  return img

#randomly flips image based on boolean value
def horizontal_orientation(img):
  if bool(random.getrandombits(1)):
    return cv.flip(img)
  else:
    pass

#rotates image based on random angle
def rotation(img, angle):
  angle = int(random.uniform(-angle, angle))
  h, w = img.shape[:2]
  M = cv.getRotationMatrix2D((int(w/2), int(h/2)), angle, 1)
  img = cv.warpAffine(img, M, (w, h))
  return img

def grayscale(img):
  img = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
  return img

#### Using functions on an image pathway and writing them back to save in the directory as adjusted images

In [ ]:
#resizes all images to be 224px by 224px, writes them back into image directory
def resize(impath):
  img = cv.imread(impath)
  eq_image = crop(img, (224, 224))
  cv.imwrite(impath, eq_image*255)

#brightens all images to random value from 1 - 1.6, writes them back into image directory
def brighten(impath):
  img = plt.imread(impath)
  eq_image = brightness(img, 0.8, 1.6)
  cv.imwrite(impath, eq_image*255)

#rotates all images by a degree from -180 to 180, writes them back into image directory
def rotate(impath):
  img = plt.imread(impath)
  eq_image = rotation(img, 180)
  cv.imwrite(impath, eq_image*255)

#randomly flips image horizontally, writes them back into image directory
def flip(impath):
  img = plt.imread(impath)
  eq_image = horizontal_orientation(img)
  cv.imwrite(impath, eq_image*255)

def gray(impath):
  img = plt.imread(impath)
  eq_image = grayscale(img)
  cv.imwrite(impath, eq_image*255)

#### Two final functions to test any image and apply random filters on all images, then save them

In [ ]:
def one_aug(impath):
  img = Image.open(impath)
  img = img.resize((224, 224), Image.LANCZOS)
  img.save(impath, "PNG" if ".png" in impath else "JPEG")

transform = A.Compose([
    A.RandomCrop(width=256, height=256),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
])

#images enhancement (takes a while ~3-4 min), loops through all file paths, utilizes all implementation functions
def final_aug():
  for folder1 in os.listdir("content/images_full"):
    for folder2 in os.listdir("content/images_full/" + folder1):
      for element in os.listdir("content/images_full/" + folder1 + "/" + folder2):
        element = os.path.join("content/images_full/" + folder1 + "/" + folder2, element)
        one_aug(element)


#grabs random image and runs augmentation on it without writing back to directory
def test_images(impath):
  image = plt.imread(impath)
  plt.imshow(crop(rotation(image, 180), (224, 224)))


#Section 2: Machine Learning

In [ ]:
#adding data from drive
!rm -rf images_full
!unzip drive/MyDrive/images_full.zip

In [ ]:
#running augmentation on all images
final_aug()

In [ ]:
data = 'content/images_full'

In [ ]:
# Define hyperparameters to tune
learning_rates = [0.01]
batch_sizes = [16, 32]
optimizers = ['Adam']
epochs = [1]


# Set up the search space
search_space = {'learning_rate': learning_rates,
                'batch_size': batch_sizes,
                'optimizer': optimizers,
                'epochs': epochs}

# Create parameter grid
parameter_grid = ParameterGrid(search_space)

In [ ]:
model = YOLO("yolov8x-cls.pt")
counter = 0

100%|██████████| 110M/110M [00:02<00:00, 39.2MB/s]


In [ ]:
model.train(data="content/images_full", imgsz=224, epochs=25, optimizer="SGD", batch=16, cos_lr = True)

Ultralytics YOLOv8.0.164 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=classify, mode=train, model=yolov8x-cls.pt, data=content/images_full, epochs=25, patience=50, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=Fa

In [ ]:
counter = 1
!rm -rf runs/classify

# Train and evaluate the model for each combination of hyperparameters
for parameters in parameter_grid:
    model = YOLO("yolov8n-cls.pt")
    # Set hyperparameters for the model
    learning_rate = parameters['learning_rate']
    batch_size = parameters['batch_size']
    optimizer = parameters['optimizer']
    num_epochs = parameters['epochs']

    # Train and evaluate the model
    model.train(data="content/images_full", lr0 = learning_rate, batch = batch_size, optimizer = optimizer, epochs = num_epochs, patience = 10, imgsz = 224)
    df = pd.read_csv('runs/classify/train' + str(counter) + '/results.csv' if counter != 1 else 'runs/classify/train/results.csv')

    os.system("mv content/runs/classify/train" + str(counter) + " drive/MyDrive/Trained Models")

    # Save the results
    result = {'learning_rate': learning_rate,
              'batch_size': batch_size,
              'optimizer': optimizer,
              'num_epochs': num_epochs,
              'accuracy': pd.Series(df['  metrics/accuracy_top1']).tolist(),
              'val/loss': df['               val/loss'].tolist()}

    if counter == 1:
      file = open('runs/classify/train/hyperparameters.json', 'w')
    else:
      file = open('runs/classify/train' + str(counter) + '/hyperparameters.json', 'w')
    file.write(json.dumps(result))
    file.close()

    counter+=1

Ultralytics YOLOv8.0.164 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=content/images_full, epochs=1, patience=10, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=F

AttributeError: ignored

In [ ]:
# new_df = pd.DataFrame(columns = ['learning_rate', 'batch_size', 'optimizer', 'num_epochs', 'accuracy', 'val/loss'])
# !rm -rf runs/classify/train5

# for folder in os.listdir('runs/classify'):
#   file = open('runs/classify/' + folder + '/hyperparameters.json', 'r')
#   hyperparameters = json.loads(file.read())
#   new_df.loc[len(new_df)] = [hyperparameters['learning_rate'], hyperparameters['batch_size'], hyperparameters['optimizer'], hyperparameters['num_epochs'], hyperparameters['accuracy'], hyperparameters['val/loss']]
#   file.close()








In [ ]:
# new_df.to_csv('drive/MyDrive/HYPERPARAMETERS.csv')